In [1]:
''' 
parser.ipynb
input: emails.csv
output: df_20.csv

1. Cut down enron file to 20% of original, selecting random entries.

2. Parse email messages into fields
New fields (columns):
email_to
email_from
cc
bcc
subject
x_from
x_to
date_sent
body
'''


' \nparser.py\ninput: emails.csv\noutput: df_20.csv\n\n1. Cut down enron file to 20% of original, selecting random entries.\n\n2. Parse email messages into fields\nNew fields (columns):\nemail_to\nemail_from\ncc\nbcc\nsubject\nx_from\nx_to\ndate_sent\nbody\n'

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import sys
import copy
import re
import email
import random


In [3]:
df = pd.read_csv('emails.csv')


In [4]:
print(df.shape)
print(df.columns)


(517401, 2)
Index(['file', 'message'], dtype='object')


In [5]:
#
# 1. Cut down enron file to 20% of original, selecting random entries.
#


In [6]:
# create df_20

# size of df_20.csv
new_size = round(int(df.shape[0] *0.20))
print(new_size)

# pick random indices
new_ind = random.sample(range(0, df.shape[0]), new_size)

# create df_20
df_20 = df.iloc[new_ind]
print(df_20.shape)
print(df_20.columns)


103480
(103480, 2)
Index(['file', 'message'], dtype='object')


In [7]:
#
# 2. Parse email messages into fields
# New fields (columns):
# email_to
# email_from
# cc
# bcc
# subject
# x_from
# x_to
# date_sent
# body
#


In [8]:
# parse email fields ['to', 'from', 'cc', 'bcc'] and put into new columns ['email_to', 'email_from', 'cc', 'bcc']

def parse_email_address(row, field):
    try:
        y = email.message_from_string(row.message)
        return email.utils.parseaddr(y.get(field))[1]
    except:
        return None
    
field_list = ['to', 'from', 'cc', 'bcc']    
new_col_list = ['email_to', 'email_from', 'cc', 'bcc']

for field, new_col in zip(field_list, new_col_list):
    print(field, new_col)
    df_20[new_col] = df_20.apply(parse_email_address, axis = 1, args = [field])
    print(df_20[new_col].describe())
    print(df_20[new_col].iloc[0:10])


to email_to


C:\Users\ibshi\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


count     103480
unique     11128
top             
freq        4369
Name: email_to, dtype: object
307592           kay.mann@enron.com
377306    benjamin.rogers@enron.com
211179        john.nowlan@enron.com
258824     louise.kitchen@enron.com
502896            lou.pai@enron.com
104112       gary.stadler@enron.com
510109      chris.mallory@enron.com
440768      airam.arteaga@enron.com
451619      david.forster@enron.com
114270        joe.capasso@enron.com
Name: email_to, dtype: object
from email_from
count                 103480
unique                 10307
top       kay.mann@enron.com
freq                    3387
Name: email_from, dtype: object
307592    jeremiahdeberry@paulhastings.com
377306         richard.broussard@enron.com
211179            vince.kaminski@enron.com
258824           dorie.hitchcock@enron.com
502896            david.delainey@enron.com
104112            david.baumbach@enron.com
510109           grace.rodriguez@enron.com
440768            hunter.shively@enron.com
4516

In [9]:
print(df_20.shape)
print(df_20.columns)


(103480, 6)
Index(['file', 'message', 'email_to', 'email_from', 'cc', 'bcc'], dtype='object')


In [10]:
# parse text fields (excluding body) ['subject', 'x-from', 'x-to'], and put into new columns ['subject', 'x_from', 'x_to']

def parse_email(row, field):
    try:
        y = email.message_from_string(row.message)
        return y.get(field)
    except:
        return None

field_list = ['subject', 'x-from', 'x-to']    
new_col_list = ['subject', 'x_from', 'x_to']

for field, new_col in zip(field_list, new_col_list):
    print(field, new_col)
    df_20[new_col] = df_20.apply(parse_email, axis = 1, args = [field])
    print(df_20[new_col].describe())
    print(df_20[new_col].iloc[0:10])


subject subject


C:\Users\ibshi\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


count     103480
unique     63467
top             
freq        3882
Name: subject, dtype: object
307592              Turbine Purchase Transaction  Documents
377306                                  Wheatland Test Data
211179                                Iraq and Crude Output
258824                                  BEAVER CREEK BIKING
502896         MTG: How to Handle Utility Default Proposals
104112                                             New book
510109             Jerald Surface Interview, July 6th, 2001
440768    Dinner Invitation - April 10, 2001 (For Tradin...
451619                                           Re: Mexico
114270                                    Positions for 9/7
Name: subject, dtype: object
x-from x_from
count       103472
unique       13274
top       Kay Mann
freq          3312
Name: x_from, dtype: object
307592    "DeBerry, Jeremiah A." <JeremiahDeberry@paulha...
377306                                    Richard Broussard
211179                              

In [11]:
print(df_20.shape)
print(df_20.columns)


(103480, 9)
Index(['file', 'message', 'email_to', 'email_from', 'cc', 'bcc', 'subject',
       'x_from', 'x_to'],
      dtype='object')


In [12]:
# parse date fields ['date'], and put into new columns ['date_sent']

def parse_date(row, field):
    try:
        y = email.message_from_string(row.message)
        date_sent_str = y.get(field)
        date_sent = email.utils.parsedate_to_datetime(date_sent_str)
        return date_sent
    except:
        return None

field_list = ['date']    
new_col_list = ['date_sent']

for field, new_col in zip(field_list, new_col_list):
    print(field, new_col)
    df_20[new_col] = df_20.apply(parse_date, axis = 1, args = [field])
    print(df_20[new_col].describe())
    print(df_20[new_col].iloc[0:10])


date date_sent
count                        103480
unique                        78806
top       2001-06-27 16:02:00-07:00
freq                            230
Name: date_sent, dtype: object
307592    2001-04-09 13:56:00-07:00
377306    2000-09-28 07:48:00-07:00
211179    2000-06-19 05:27:00-07:00
258824    2001-08-13 10:45:08-07:00
502896    2000-11-29 07:18:00-08:00
104112    2000-08-17 06:42:00-07:00
510109    2001-06-29 12:02:27-07:00
440768    2001-03-21 01:43:00-08:00
451619    2000-07-06 09:06:00-07:00
114270    2001-09-06 14:25:26-07:00
Name: date_sent, dtype: object


C:\Users\ibshi\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
# parse body field and put into new column ['body']


def parse_body(row):
    try:
        y = email.message_from_string(row.message)
        if y.is_multipart():
            for part in y.walk():
                ctype = part.get_content_type()
                cdispo = str(part.get('Content-Disposition'))

                # skip any text/plain (txt) attachments
                if ctype == 'text/plain' and 'attachment' not in cdispo:
                    body = part.get_payload(decode=True)  # decode
                    break
        # not multipart - i.e. plain text, no attachments, keeping fingers crossed
        else:
            body = y.get_payload(decode=True)
        return body
    except:
        return None

df_20['body'] = df_20.apply(parse_body, axis = 1)
print(df_20['body'].describe())
print(df_20['body'].iloc[0:3])


count                                                103480
unique                                                84495
top       b"As you know, Enron Net Works (ENW) and Enron...
freq                                                     30
Name: body, dtype: object
307592    b'"paulhastings.com" made the following annota...
377306    b"I verified the Wheatland test data in the re...
211179    b"An interesting piece of information from the...
Name: body, dtype: object


C:\Users\ibshi\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
print(df_20.shape)
print(df_20.columns)


(103480, 11)
Index(['file', 'message', 'email_to', 'email_from', 'cc', 'bcc', 'subject',
       'x_from', 'x_to', 'date_sent', 'body'],
      dtype='object')


In [15]:
# save df_20.csv

df_20.to_csv('df_20.csv', index = True)
